# O intuito desse notebook e recuperar todas as imagens disponiveis  sobre pneumonia e covid e disponibilizar em um banco de dados


## bibliotecas necessárias

In [3]:
!pip install pydicom

In [20]:
from imutils import paths
import pydicom as dicom
import pandas as pd
import shutil
import cv2 
import os

##  diretorios target e source

In [21]:
def getTargetDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [22]:
dir_target_diretorio_raiz = getTargetDiretorio("MaquinaLocal")
dir_target_diretorio_dataset =r"{0}/{1}".format(dir_target_diretorio_raiz,'dataset')
dir_target_diretorio_imagem_covid =r"{0}/{1}".format(dir_target_diretorio_dataset,'covid')
dir_target_diretorio_pneumonia_normal =r"{0}/{1}".format(dir_target_diretorio_dataset,'normal')
dir_target_diretorio_pneumonia_pneumonia =r"{0}/{1}".format(dir_target_diretorio_dataset,'pneumonia')

dir_source_imagens_Covid = []

dir_source_imagens_Covid_cohen =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/images'

dir_source_imagens_Covid.append(dir_source_imagens_Covid_cohen)


## diretorios pra pneumonia
dir_source_rsna_pneumonia =r'D:/Repos/rsna-pneumonia-detection-challenge'
dir_source_pneumonia_csv_detailed_class =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_detailed_class_info.csv')
dir_source_pneumonia_csv_train_labels = r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_labels.csv') 

dir_source_pneumonia_detailed_class_imagens =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_images')




print("source")
print(dir_source_imagens_Covid)
print(dir_source_pneumonia_detailed_class_imagens)


print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)






source
['D:/Repos/datasetsGitHub/covid-chestxray-dataset/images']
D:/Repos/rsna-pneumonia-detection-challenge/stage_2_train_images
target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia


In [23]:
lista_diretorios_target=[]
lista_diretorios_target.append(dir_target_diretorio_raiz)
lista_diretorios_target.append(dir_target_diretorio_dataset),
lista_diretorios_target.append(dir_target_diretorio_imagem_covid),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_normal),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_pneumonia)

for item in lista_diretorios_target:
    if not os.path.exists(item):
        os.mkdir(item)
    




## metodos auxiliares

In [24]:
def clear_diretorio(diretorio):
    for file in os.listdir(diretorio):
        full_name =os.path.join(diretorio, file)
        
        if os.path.exists(full_name):
            os.remove(full_name) 
        
        

def get_imagens_covid(dataframe,diretorio):
        lista =[]
        for (i,row) in dataframe.iterrows():
                imagemName=row["filename"]
                image_source  = "{0}/{1}".format(diretorio,imagemName)
                lista.append(image_source)

        return lista


def get_imagens_diretorio(diretorio):
        imagePaths =[]
        imagePaths = list(paths.list_images(diretorio))
        imagePaths = [ item.replace("\\","/") for item in imagePaths]
        
        return imagePaths    


def move_imagens_from_source_to_target(source_lista,target):
    for source_path in source_lista:
        imagemName =source_path.split('/')[-1]
        source_target= "{0}/{1}".format(target,imagemName)
    # print(source_path)
    # print(source_target)
        if(os.path.exists(source_path)):
            if not os.path.exists(source_target):
                shutil.copy2(source_path,source_target)
    
                


## leitura dos repositorios

In [25]:
source_dir_metadata_local_cohen  = r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/metadata.csv'

metadata_web_git ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'



In [26]:
df_git_covid_web  = pd.read_csv(metadata_web_git,error_bad_lines=False)

df_git_covid_local = pd.read_csv(source_dir_metadata_local_cohen,error_bad_lines=False)


print(len(df_git_covid_web))
print(len(df_git_covid_local))

if(len(df_git_covid_web) > len(df_git_covid_local)):
    print('temos {0} imagens com covid a mais no git web'.format(len(df_git_covid_web) - len(df_git_covid_local)))
elif(len(df_git_covid_web) < len(df_git_covid_local)):
    print('foram removidas {0} imagens  no git web'.format(len(df_git_covid_local) - len(df_git_covid_web)))


682
565
temos 117 imagens com covid a mais no git web


In [38]:
map_prognostico  =  { 'Lung Opacity': 'pneumonia',
    'Normal' :'normal',
    'COVID-19':'COVID-19',
    'SARS': 'pneumonia',
    'MERS': 'pneumonia',
    'Streptococcus': 'pneumonia',
    'Klebsiella': 'pneumonia',
    'Chlamydophila': 'pneumonia',
    'Legionella': 'pneumonia',
    'Normal': 'normal'

}





In [40]:
quantidade_registros = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

## caso necessario limpa os diretorios targets

In [29]:
clear_diretorio(dir_target_diretorio_imagem_covid)     
clear_diretorio(dir_target_diretorio_pneumonia_pneumonia)       
clear_diretorio(dir_target_diretorio_pneumonia_normal)       

## tranferencia dos arquivos de Covid 19

In [41]:
df_git_covid_local.shape

(565, 28)

In [42]:
df_git_covid_local['prognostico'] = df_git_covid_local['finding'].map(map_prognostico)


# df_covid =  df_metadata.query('indicadorPrognostico == 1 and posicao_imagem =="PA" and tipoExame =="X-ray"')

In [43]:
df_git_covid_local['prognostico'].value_counts()

COVID-19     420
pneumonia     40
Name: prognostico, dtype: int64

In [ ]:
for row in df_rsna[["prognostico","patientId"]].iterrows():
    # imagem_name = row[1].patientId
    # prognostico = row[1].prognostico
    # source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    # if os.path.exists(source_imagem):
    #     dir_target =""
    #     if(prognostico =="normal"):
    #         dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
    #     elif (prognostico =="pneumonia"):
    #         dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
    #     if not os.path.exists(dir_target):
    #         ds = dicom.dcmread(source_imagem)
    #         pixel_array_numpy = ds.pixel_array
    #         cv2.imwrite(dir_target, pixel_array_numpy)
    #         quantidade_registos[prognostico] +=1

In [ ]:
for row in df_rsna[["prognostico","patientId"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
        elif (prognostico =="pneumonia"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
        if not os.path.exists(dir_target):
            ds = dicom.dcmread(source_imagem)
            pixel_array_numpy = ds.pixel_array
            cv2.imwrite(dir_target, pixel_array_numpy)
            quantidade_registos[prognostico] +=1

## tranfere as imagens do repositorio do rsna para o dataset

In [11]:
df_csv_detailed_class = pd.read_csv(dir_source_pneumonia_csv_detailed_class)

df_csv_detailed_class["class"].value_counts()



No Lung Opacity / Not Normal    11821
Lung Opacity                     9555
Normal                           8851
Name: class, dtype: int64

In [12]:
df_rsna = df_csv_detailed_class.loc[df_csv_detailed_class['class'].isin(['Normal','Lung Opacity'])]

In [13]:
df_rsna["prognostico"] = df_rsna['class'].map(map_prognostico)

df_rsna["class"].value_counts()

Lung Opacity    9555
Normal          8851
Name: class, dtype: int64

In [45]:


for row in df_rsna[["prognostico","patientId"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
        elif (prognostico =="pneumonia"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
        if not os.path.exists(dir_target):
            ds = dicom.dcmread(source_imagem)
            pixel_array_numpy = ds.pixel_array
            cv2.imwrite(dir_target, pixel_array_numpy)
            quantidade_registos[prognostico] +=1
            



print(quantidade_registos)        


{'normal': 1, 'pneumonia': 0, 'COVID-19': 0}


In [14]:
df_covid  = pd.read_csv(source_dir_metadata_local)

In [15]:
df_covid.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
